In [1]:
# load the packages
import json  
import re
from collections import Counter
import copy
import string
import time
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize  
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from textblob import TextBlob
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.tag import StanfordNERTagger
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

from sklearn.metrics import classification_report

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


# Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

SEED = 42

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jiaying/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jiaying/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jiaying/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jiaying/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('desc_clean.csv', lineterminator='\n')

In [3]:
df.description.fillna('', inplace = True)

In [4]:
# count vectorizer
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df.description)
print(dtm_tf.shape)

(120746, 46433)


In [5]:
dtm_tf_df = pd.DataFrame(dtm_tf.todense(), columns=tf_vectorizer.get_feature_names())

In [11]:
s = dtm_tf_df.sum().sort_values(ascending = False)
common_words = s[:100].index

In [12]:
common_words

Index(['video', 'music', 'new', 'twitter', 'videos', 'facebook', 'channel',
       'subscribe', 'instagram', 'youtube', 'like', 'news', 'watch',
       'official', 'live', 'follow', 'just', 'make', 'love', 'use', 'late',
       'know', 'time', 'world', 'dont', 'best', 'website', 'night', 'latest',
       'episode', 'play', 'day', 'life', 'jimmy', 'production', 'want',
       'available', 'check', 'entertainment', 'free', 'people', 'film',
       'links', 'song', 'episodes', 'movie', 'social', 'exclusive', 'way',
       'star', 'voice', 'got', 'box', 'google', 'album', 'today', 'click',
       'series', 'code', 'game', 'media', 'thanks', 'food', 'producer',
       'family', 'content', 'link', 'real', 'tumblr', 'merch', 'james', 'home',
       'things', 'try', 'original', 'nbc', 'look', 'kimmel', 'visit', 'sports',
       'favorite', 'think', 'big', 'cbs', 'season', 'youre', 'director',
       'makeup', 'stream', 'performing', 'come', 'tmz', 'spotify', 'talk',
       'story', 'support', 

In [19]:
customized_stopwords = stop_words.union(set(common_words))

In [25]:
# tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                    stop_words = customized_stopwords,
                                    lowercase = True,
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    max_df = 0.5, 
                                    min_df = 10)
dtm_tfidf = tfidf_vectorizer.fit_transform(df.description)
print(dtm_tfidf.shape)

(120746, 46492)


In [26]:
# lda
lda_tf = LatentDirichletAllocation(n_components=20, random_state=SEED)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=SEED)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [28]:
# topic modeling
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8      0.033975 -0.096909       1        1  7.489021
6      0.039875 -0.056921       2        1  6.117605
16     0.013230  0.087831       3        1  5.789802
7      0.047579 -0.014543       4        1  5.767875
10     0.014439 -0.019031       5        1  5.298225
0      0.030969 -0.075091       6        1  5.177651
5      0.032314 -0.109717       7        1  5.110906
19     0.047537  0.008642       8        1  5.104755
1      0.037901 -0.033889       9        1  5.082036
13    -0.319503 -0.054274      10        1  4.870808
11     0.027306 -0.033983      11        1  4.804677
12    -0.017118 -0.027187      12        1  4.784781
9      0.045482  0.011564      13        1  4.609344
3     -0.018024  0.151559      14        1  4.598882
17     0.025204  0.021245      15        1  4.556581
2      0.027552  0.017850      16        1  4.402317
4     -0.025809 -0.056084      17        1  4.288797
18    -0.019165  0.037316      18        1  4.078059
15     0.006347  0.080963      19        1  4.034425
14    -0.030091  0.160659      20        1  4.033454, topic_info=      Category        Freq      Term       Total  loglift  logprob
27895  Default  379.000000   netflix  379.000000  30.0000  30.0000
23443  Default  695.000000    listen  695.000000  29.0000  29.0000
36444  Default  318.000000      seth  318.000000  28.0000  28.0000
45586  Default  304.000000       wwe  304.000000  27.0000  27.0000
41680  Default  395.000000   tonight  395.000000  26.0000  26.0000
...        ...         ...       ...         ...      ...      ...
27137  Topic20   44.527555      much  409.498947   0.9917  -6.2816
43177  Topic20   40.780723       usa  117.413060   2.1531  -6.3695
44511  Topic20   41.244789  watching  451.315680   0.8179  -6.3582
33544  Topic20   40.692180   records  580.391332   0.5529  -6.3717
20765  Topic20   39.806397      john  342.727691   1.0576  -6.3937

[1380 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
20        17  0.958940   aarons
37         1  0.007326      abc
37         2  0.036632      abc
37         3  0.058611      abc
37         4  0.109896      abc
...      ...       ...      ...
46348     10  0.986943  zeigler
46452      9  0.915067  zombies
46452     17  0.048161  zombies
46465     14  0.035087     zoya
46465     18  0.947349     zoya

[7258 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 17, 8, 11, 1, 6, 20, 2, 14, 12, 13, 10, 4, 18, 3, 5, 19, 16, 15])